In [1]:
####### 对微博进行相似句子Top-5查询######
#导入相关包#
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re
import os

#读取文件#
os.chdir('D:\\开课吧数据挖掘资料\\第六讲—名企—近邻邻域查找与youtub推荐\\L6')
f = open('./weibos.txt', 'r', encoding='UTF-8')
text = f.read()

In [3]:
##数据处理##

# 以#号 句号，双引号，叹号，问号作为分隔，去掉\n换行符号 切割为句子
sentences_raw = re.split('[# 。” ！？]',text.replace('\n', ''))

# 如果为空，则删除.即保留有值的字段
sentences=[]
for i in sentences_raw:
    if len(i)>0:
        sentences.append(i)        

In [5]:
##macro##

# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

# 设置停用词 暂无停用词
stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]

In [6]:
## 微博文档 创建LSH Forest及MinHash对象 运行##

# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
    
# index所有key，以便可以进行检索   
forest.index()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.316 seconds.
Prefix dict has been built successfully.


In [11]:
##查询语句运行，输出原文档top 5相似语句##

query = '斯科拉里水平不行'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 5)
print("*"*50+"斯科拉里水平不行,Top 5相似语句输出"+"*"*50)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))

**************************************************斯科拉里水平不行,Top 5相似语句输出**************************************************
0 0.2890625 斯科拉里愿意执教国足
33 0.421875 大家支持斯科拉里不
16 0.625 斯科拉里的水平，还不如里皮
17 0.2890625 斯科拉里，看好的不是国足，而是年薪……
31 0.1640625 ​带队结果绝不会比里皮更差，
